In [1]:
import numpy as np
import math
import os.path
import csv


In [2]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input

Using TensorFlow backend.
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
from keras.models import Model, load_model
import cv2

In [4]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

In [5]:
from scipy.spatial import distance

In [6]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [7]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [8]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [9]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [13]:
def getFrame(sec):
    start = 120000
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*500)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [11]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [14]:
# main program
from scipy.spatial import distance

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
data = []
labels = []
respondent = []
for j in range(60):
    for i in range(11):
        if j == 55 or j == 58:
            vidcap = cv2.VideoCapture('Fold5_part2/Fold5_part2/' + str(j) +'/' + str(i) + '.mp4')
        else:
            vidcap = cv2.VideoCapture('Fold5_part2/Fold5_part2/' + str(j) +'/' + str(i) + '.MOV')
        sec = 0
        frameRate = 1
        success, image  = getFrame(sec)
        count = 0
        
        while success and count < 400:
            #print("not detected")
            image = image_resize(image,width = 720 ,height = 1080)
            #หมุนวิดิโอ
            if j < 60 and j > 55:
                if j == 58:
                    image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                else:
                    image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            cv2.imshow('image',image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if len(faces) == 0:

                sec = sec + frameRate
                sec = round(sec, 2)
                success, image = getFrame(sec)
                print("not detected")
            else:
                landmarks = extract_face_landmarks(gray)
                if landmarks is not None:
                    if sum(sum(landmarks)) != 0:
                        #print(landmarks)
                        count += 1
                        data.append(landmarks)
                        labels.append([i])
                        respondent.append([j])

                        sec = sec + frameRate
                        sec = round(sec, 2)
                        success, image = getFrame(sec)
                        print(count)

                else:  
                    sec = sec + frameRate
                    sec = round(sec, 2)
                    success, image = getFrame(sec)
                    print("not detected")
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
1
2
3
4


not detected
not detected
not detected
not detected
not detected
not detected
5
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
6
7
8
not detected
9
10
not detected
11
not detected
12
not detected
13
14
15
16
17
18
not detected
19
20
21
not detected
22
not detected
not detected
23
24
not detected
not detected
not detected
not detected
not detected
25
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not detected
not de

In [15]:
data = np.array(data)
labels = np.array(labels)

In [16]:
data

array([[[307, 143],
        [308, 157],
        [310, 170],
        ...,
        [364, 180],
        [360, 181],
        [355, 180]],

       [[309, 154],
        [311, 167],
        [313, 180],
        ...,
        [368, 189],
        [363, 190],
        [359, 189]],

       [[312, 149],
        [312, 163],
        [314, 176],
        ...,
        [368, 187],
        [363, 188],
        [358, 187]],

       ...,

       [[105, 347],
        [103, 371],
        [106, 394],
        ...,
        [189, 440],
        [180, 440],
        [170, 439]],

       [[102, 349],
        [101, 372],
        [103, 395],
        ...,
        [184, 440],
        [174, 440],
        [163, 438]],

       [[ 99, 348],
        [ 97, 370],
        [ 99, 394],
        ...,
        [179, 438],
        [169, 439],
        [160, 437]]])

In [17]:
labels

array([[ 0],
       [ 0],
       [ 0],
       ...,
       [10],
       [10],
       [10]])

In [18]:
respondent

[[55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],
 [55],

In [19]:
features = []
i = 0
for d in data:
    eye = d[36:68]
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    cir = circularity(eye)
    mouth_eye = mouth_over_eye(eye)
    features.append([ear, mar, cir, mouth_eye,labels[i][0],respondent[i][0]])
    i+=1

In [20]:
from pandas import DataFrame
pd_feature = DataFrame(features,columns=['EAR','MAR','Circularity','MOE','Y','Respondent'])

In [ ]:
features = np.array(features)
features.shape

In [ ]:
features

In [21]:
pd_feature

,EAR,MAR,Circularity,MOE,Y,Respondent
0,0.341632,1.036523,0.558062,3.034030,0,55
1,0.368421,0.964764,0.614811,2.618645,0,55
2,0.289074,1.000000,0.436269,3.459327,0,55
3,0.344251,1.072587,0.475702,3.115707,0,55
4,0.364822,1.348893,0.465361,3.697404,0,55
...,...,...,...,...,...,...
4865,0.275401,1.182630,0.414005,4.294205,10,59
4866,0.204631,1.189241,0.291487,5.811628,10,59
4867,0.290996,1.110189,0.398633,3.815133,10,59
4868,0.301910,1.143009,0.420446,3.785924,10,59


In [23]:
pd_feature = pd_feature[pd_feature.Respondent != 58]

In [24]:
pd_feature

,EAR,MAR,Circularity,MOE,Y,Respondent
0,0.341632,1.036523,0.558062,3.034030,0,55
1,0.368421,0.964764,0.614811,2.618645,0,55
2,0.289074,1.000000,0.436269,3.459327,0,55
3,0.344251,1.072587,0.475702,3.115707,0,55
4,0.364822,1.348893,0.465361,3.697404,0,55
...,...,...,...,...,...,...
4865,0.275401,1.182630,0.414005,4.294205,10,59
4866,0.204631,1.189241,0.291487,5.811628,10,59
4867,0.290996,1.110189,0.398633,3.815133,10,59
4868,0.301910,1.143009,0.420446,3.785924,10,59


In [25]:
pd_feature.to_csv('Fold5_part2_features.csv',index=False)

In [ ]:
data.shape

In [ ]:
labels.shape

In [ ]:
# change file name to save 
#np.save(open('Data_60_10.npy', 'wb'),data)
#np.save(open('Fold5_part2_features_60_10.npy', 'wb'),features)
#np.save(open('Fold5_part2_labels_60_10.npy', 'wb'),labels)
#np.savetxt("Fold5_part2_features.csv", features, delimiter = ",")
#np.savetxt("Fold5_part2_labels.csv", labels, delimiter = ",")